# Data Quality Checks
- Check data is empty or not

In [ ]:
import configparser
from s3path import S3Path


# ***** Access AWS Cloud configure ************
config = configparser.ConfigParser()
config.read_file(
    open('/Users/oneforall_nick/workspace/Udacity_capstone_project/cfg/dl.cfg'))

aws_access_key = config["ACCESS"]["AWS_ACCESS_KEY_ID"]
aws_secret_access_key = config["ACCESS"]["AWS_SECRET_ACCESS_KEY"]
aws_token = config["ACCESS"]["AWS_TOKEN"]
dest_aws_s3_bucket = config["S3"]["DEST_S3_BUCKET"]


In [ ]:
dim_bucket_path = S3Path(f"{dest_aws_s3_bucket}/dimension_table")
fact_bucket_path = S3Path(f"{dest_aws_s3_bucket}/fact_table")

In [ ]:
# ******** Dimension Tables ********
# Data Object Name: df_immigration_personal
if data_list := [str(path_1).split('/')[-1] for path in dim_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if str(path_1).split('/')[-2] in 'df_immigration_personal']:
    print(
        f"Data: df_immigration_personal, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")

# Data Object Name: imm_address
if data_list := [str(path_1).split('/')[-1] for path in dim_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if str(path_1).split('/')[-2] in 'imm_address']:
    print(f"Data: imm_address, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")


# Data Object Name: imm_city_res_label
if data_list := [str(path_1).split('/')[-1] for path in dim_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if str(path_1).split('/')[-2] in 'imm_city_res_label']:
    print(f"Data: imm_city_res_label, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")


# Data Object Name: imm_destination_city
if data_list := [str(path_1).split('/')[-1] for path in dim_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if str(path_1).split('/')[-2] in 'imm_destination_city']:
    print(f"Data: imm_destination_city, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")


# Data Object Name: imm_travel_code
if data_list := [str(path_1).split('/')[-1] for path in dim_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if str(path_1).split('/')[-2] in 'imm_travel_code']:
    print(f"Data: imm_travel_code, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")


# Data Object Name: imm_visa
if data_list := [str(path_1).split('/')[-1] for path in dim_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if str(path_1).split('/')[-2] in 'imm_visa']:
    print(f"Data: imm_visa, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")


# Data Object Name: immigration_main_information
if data_list := [str(path_1) for path in dim_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if path_1.is_dir() if str(path_1).split('/')[-2] in 'immigration_main_information']:
    print(
        f"Data: immigration_main_information, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")


# Data Object Name: news_article_data
if data_list := [str(path_1).split('/')[-1] for path in dim_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if str(path_1).split('/')[-2] in 'news_article_data']:
    print(
        f"Data: news_article_data, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")


# Data Object Name: us_cities_demographics_data
if data_list := [str(path_1).split('/')[-1] for path in dim_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if str(path_1).split('/')[-2] in 'us_cities_demographics_data']:
    print(
        f"Data: us_cities_demographics_data, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")


In [ ]:
# ******** Fact Tables ********
# Data Object Name: df_immigration_personal
if data_list := [str(path_1).split('/')[-1] for path in fact_bucket_path.iterdir() if path.is_dir() for path_1 in path.iterdir() if path_1.is_dir() if str(path_1).split('/')[-2] in 'notification']:
    print(f"Data: notification, Count data objects: {len(data_list)}")
else:
    raise ValueError("This table does not contain data!!")
